In [6]:
import os

In [7]:
pwd

'/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/research'

In [11]:
import os
os.chdir("../")

In [12]:
%pwd

'/Users/satwik/Downloads/MLproj/airlines_sentiment_classification'

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    training_data_path : Path
    training_data_file : Path
    training_cleansed_data : Path
    # base_model_path: Path
    # updated_base_model_path: Path
    # # params_image_size: 
    # num_train_epochs: int
    # per_device_train_batch_size: int
    # per_device_eval_batch_size: int
    # warmup_steps: int
    # weight_decay: float
    # # logging_dir: Path
    # max_steps: int
    # save_steps: int
    # logging_steps: int



In [14]:
from pathlib import Path

CONFIG_FILE_PATH = Path("/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/config/config.yaml")
PARAMS_FILE_PATH = Path("/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/params.yaml")

In [15]:
from src.airlinesSentiment.constants import *
from src.airlinesSentiment.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    
    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        config = self.config.feature_engineering

        create_directories([config.root_dir])

        data_preprocessing_config = DataPreprocessingConfig(
             root_dir=config.root_dir,
             training_data_path=config.training_data_path,
             training_data_file=config.training_data_file,
             training_cleansed_data=config.training_cleansed_data
        )

        return data_preprocessing_config


        # prepare_base_model_config = PrepareBaseModelConfig(
        #     root_dir=Path(config.root_dir),
        #     base_model_path=Path(config.base_model_path),
        #     updated_base_model_path=Path(config.updated_base_model_path),
        #     num_train_epochs=self.params.EPOCH,
        #     per_device_train_batch_size=self.params.TRAIN_BATCH_SIZE,
        #     per_device_eval_batch_size=self.params.EVAL_BATCH_SIZE,
        #     warmup_steps=self.params.WARMUP_STEPS,
        #     weight_decay=self.params.WEIGHT_DECAY,
        #     max_steps=self.params.MAX_STEPS,
        #     save_steps=self.params.SAVE_STEPS,
        #     logging_steps=self.params.LOGGIN_STEPS
        # )

        # return prepare_base_model_config

In [ ]:
import os
from airlinesSentiment import logger
from airlinesSentiment.utils.common import get_size
import gdown
import spacy
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from nltk.corpus import stopwords
import string
import pandas as pd
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

logger = logging.getLogger(__name__)



class 




In [ ]:
# class DataPreprocessing:
#     def __init__(self, config: Datapreprosessing):
#         self.config = config

In [ ]:
# class PrepareBaseModel:
#     def __init__(self, config: PrepareBaseModelConfig):
#         self.config = config